# Adaboost Implementation


#### **1. Combined Classifier Prediction**
The final strong classifier \(H(x)\) combines the predictions of \(M\) weak learners, weighted by their importance (\(\alpha_m\)):
$$
H(x) = \text{sign}\left(\sum_{m=1}^M \alpha_m \cdot h_m(x)\right).
$$
Inference is considered from the sign of H(x) where all the multiple weak learners are used 
#### **2. Exponential Loss Function**
AdaBoost minimizes the exponential loss function, which assigns higher penalties to misclassified samples:
$$
L = \sum_{i=1}^n w_i \cdot \exp(-y_i \cdot H(x_i)),
$$
where:
- \(w_i\): Weight of sample \(i\),
- \(y_i\): True label of sample \(i\),
- \(H(x_i)\): Combined prediction from all weak learners for sample \(i\).

#### **3. Minimizing the Loss**
To find the weight of the current weak learner (\(\alpha_m\)), the exponential loss is minimized with respect to \(\alpha_m\):
$$
\alpha_m = 0.5 \cdot \ln\left(\frac{1 - \text{error}}{\text{error}}\right),
$$
where \(\text{error}\) is the weighted classification error of the weak learner:
$$
\text{error} = \frac{\sum_{i=1}^n w_i \cdot \mathbb{I}(h_m(x_i) \neq y_i)}{\sum_{i=1}^n w_i}.
$$

#### **4. Sample Weights Update**
After determining \(\alpha_m\), the sample weights are updated to emphasize misclassified samples:
$$
w_i \leftarrow w_i \cdot \exp(-\alpha_m \cdot y_i \cdot h_m(x_i)).
$$
- **Correctly classified samples**: \(y_i \cdot h_m(x_i) = +1\), weight decreases.
- **Misclassified samples**: \(y_i \cdot h_m(x_i) = -1\), weight increases.

#### **5. Normalization**
To ensure the weights remain a valid probability distribution:
$$
w_i \leftarrow \frac{w_i}{\sum_{j=1}^n w_j}.
$$
This ensures:
$$
\sum_{i=1}^n w_i = 1.
$$


### Final Notes
- AdaBoost iteratively builds a strong classifier by focusing on harder-to-classify samples through weight adjustments.
- The combined classifier \(H(x)\) is a weighted majority vote of the weak learners.



#### *** Derivation: Minimize the Loss**
To find the weight of the current weak learner (\(\alpha_m\)), the exponential loss is minimized with respect to \(\alpha_m\):

**Step 1: Take the derivative of \(L_m\) with respect to \(\alpha_m\):**
$$
\frac{\partial L_m}{\partial \alpha_m} = -W_{\text{correct}} \cdot \exp(-\alpha_m) + W_{\text{misclassified}} \cdot \exp(+\alpha_m) = 0,
$$
where:
- \(W_{\text{correct}}\): Sum of weights for correctly classified samples,
- \(W_{\text{misclassified}}\): Sum of weights for misclassified samples.

**Step 2: Rearrange terms:**
$$
W_{\text{correct}} \cdot \exp(-\alpha_m) = W_{\text{misclassified}} \cdot \exp(+\alpha_m).
$$

**Step 3: Divide through by \(\exp(-\alpha_m)\):**
$$
W_{\text{correct}} = W_{\text{misclassified}} \cdot \exp(2\alpha_m).
$$

**Step 4: Solve for \(\exp(2\alpha_m)\):**
$$
\exp(2\alpha_m) = \frac{W_{\text{correct}}}{W_{\text{misclassified}}}.
$$

**Step 5: Take the natural logarithm:**
$$
2\alpha_m = \ln\left(\frac{W_{\text{correct}}}{W_{\text{misclassified}}}\right).
$$

**Step 6: Simplify:**
$$
\alpha_m = 0.5 \cdot \ln\left(\frac{1 - \text{error}}{\text{error}}\right),
$$
where:
$$
\text{error} = \frac{W_{\text{misclassified}}}{W_{\text{total}}}.
$$


In [3]:
import numpy as np
from collections import Counter

class Node:
    #node consists of feature, which threshold it is split on, child node left and right childe 
    # node and if it is a leaf value of the leaft node i.e class of the leaf node
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    #if value is not none then it is a leaf node will be appened inthe training state and used for interference purpose    
    def is_leaf_node(self):
        return self.value is not None



'''
grow tree 
best split
information gain
entropy 
'''

class DecisionTree:
    #splitting and stopping critieria - min samples in the node to split , max layer depth of the tree, total number of features in the tree both splitting and 
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        # Count occurrences of each class
        unique_labels, counts = np.unique(y, return_counts=True)
        ps = counts / len(y)  # Probabilities
        return -np.sum([p * np.log(p) for p in ps if p > 0])



    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        

In [ ]:
class AdaBoost:
    def __init__(self, n_estimators = 50):
        self.n_estimators =n_estimators
        self.alphas = []
        self.models = []
    
    def fit(self,X,y):
        n_samples, _ = X.shape
        weights = np.ones(n_samples)/n_samples

        for _ in range(self.n_estimators):

            #builing a weak learner decision tree stump 
            tree = DecisionTree(max_depth = 1)
            tree.fit(X,y)

            y_pred = tree.predict(X)

            #calculating the weighted loss  of this model predictions 
            error = np.sum(weights*(y_pred !=y))/np.sum(weights)

            #calculaitng this model weight alpha this is derived from above and 
            #this deriviation is from by minizing the exponential loss function consider for this 

            alpha = 0.5 * np.log( (1- error)/(error+1e-10 )) #to get ride of zero divison error
            self.alphas.append(alpha)
            self.models.append(tree)

            #updatating the weights this is the sample weights 
            #simply assinging the exponential term for the weights 
            #high weightage for more misclasifed predications and less weight assinged for rightly classifeid 
            weights *= np.exp(-alpha * y * y_pred) 
            weights /= np.sum(weights) #normlizign the updated weights to sum 1 


    def predict(self, X):

        #build the emply pred array of n samples 
        pred = np.zeros(X.shape[0])

        #finding H = a1*h1 + a2*h2 +.....+am*hm for m - boosted m weak samples 
        for alpha, model in zip(self.alphas, self.models):
            pred += alpha * model.predict(X)
        
        return np.sign(pred)  #returns +1 or -1 from the classification 

            




In [10]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

# Create a simple dataset
X, y = make_classification(n_samples=100, n_features=5, random_state=42)
y = np.where(y == 0, -1, 1)  # Convert labels to -1 and 1 for binary classification

# Split into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# AdaBoost
adaboost = AdaBoost(n_estimators=50)
adaboost.fit(X_train, y_train)
y_pred_ada = adaboost.predict(X_test)
print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred_ada))


AdaBoost Accuracy: 1.0
